In [1]:
from __future__ import print_function
import logging
import numpy as np
import sys
import csv
import json

from optparse import OptionParser
from configparser import ConfigParser
import time
from pymongo import MongoClient
from twython import Twython, TwythonRateLimitError
from dateutil import parser
import datetime as dt
from datetime import datetime

from twitter import Twitter, OAuth
from pandas import DataFrame, read_csv
import pandas as pd

config = ConfigParser()
config.read('twitter.cfg')
# for py27 change config to get_config
APP_KEY       = config.get('credentials','app_key')
APP_SECRET    = config['credentials']['app_secret']
twitter       = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN  = twitter.obtain_access_token()
twitter       = Twython(APP_KEY, access_token=ACCESS_TOKEN)

logging.basicConfig(filename='Twitter_Brexit.log',level=logging.DEBUG)
logging.info('TWEETS_BREXIT')
logging.warning('GO!')

screen_name  = 'retromaquinitas'     # The main twitter account
n_days       = 30          # Only tweets more recent than n_days are kept
n_friends  = 500

def wait_for_awhile():
    reset = int(twitter.get_lastfunction_header('x-rate-limit-reset'))
    wait = max(reset - time.time(), 0) + 10
    print("Rate limit exceeded waiting: %sm %0.0fs"%
            (int(int( wait)/60),wait % 60 ))
    time.sleep(wait)
    


In [33]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [3]:
result = twitter.lookup_status(id='730735231712538624')

for tw in result:
    print(tw["id"])
    print(tw["user"])
    print(tw)

730735231712538624
{'id': 2365445227, 'id_str': '2365445227', 'name': 'European Mainstream', 'screen_name': 'EU_Mainstream', 'location': 'United Kingdom', 'description': "Mainstream of Conservative Party, united by belief that UK's national interest is best served working for changes within EU |\n\n\nFB page: https://t.co/3dpgYhbNZL", 'url': 'http://t.co/5ww1lJ0myT', 'entities': {'url': {'urls': [{'url': 'http://t.co/5ww1lJ0myT', 'expanded_url': 'http://www.europeanmainstream.org.uk', 'display_url': 'europeanmainstream.org.uk', 'indices': [0, 22]}]}, 'description': {'urls': [{'url': 'https://t.co/3dpgYhbNZL', 'expanded_url': 'https://www.facebook.com/ConservativeEuropeamMainstream/', 'display_url': 'facebook.com/ConservativeEu…', 'indices': [137, 160]}]}}, 'protected': False, 'followers_count': 1144, 'friends_count': 681, 'listed_count': 54, 'created_at': 'Fri Feb 28 11:00:22 +0000 2014', 'favourites_count': 434, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': 

In [ ]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('Brexit_tweets_stance') if isfile(join('Brexit_tweets_stance', f))]

onlyfiles

In [41]:
filelist = list(chunks(onlyfiles, 100))[0]

In [27]:
df = pd.read_csv('Brexit_tweets_stance/'+ onlyfiles[0], sep='\t', header = None, names = ['id', 'sentiment'])
df.head()

,id,sentiment
0,730759428430008320,Positive
1,730759413938655232,Negative
2,730759400445644801,Positive
3,730759393965412352,Neutral
4,730759392925261824,Positive


In [46]:
df_from_each_file = (pd.read_csv('Brexit_tweets_stance/' + f, sep='\t', header = None, names = ['id', 'sentiment']) for f in filelist)
concatenated_df   = pd.concat(df_from_each_file, sort = False)

In [49]:
concatenated_df.shape

(164134, 2)

In [50]:
list(chunks(concatenated_df["id"].values, 100))[0]

array([730759428430008320, 730759413938655232, 730759400445644801,
       730759393965412352, 730759392925261824, 730759383253192705,
       730759377548918784, 730759375174897664, 730759372603789313,
       730759366530478080, 730759365142167552, 730759363175034880,
       730759346766876672, 730759344879505409, 730759334796349440,
       730759321068445696, 730759317884915712, 730759317075439617,
       730759310754611201, 730759308707811328, 730759305792749568,
       730759304253452289, 730759296955367424, 730759295478960129,
       730759289057513472, 730759278630424577, 730759275543470080,
       730759264386617344, 730759262226485248, 730759258241945600,
       730759256530661377, 730759251883347968, 730759244979396608,
       730759228223279104, 730759220631621632, 730759212507090944,
       730759212268052480, 730759212163305472, 730759209579646976,
       730759204747788288, 730759202059259904, 730759201472036864,
       730759190042558464, 730759188973006848, 730759186997448

In [30]:
#  MongoDB connection
client      = MongoClient()
db = client.brexit
collection = db.tweets

In [37]:
",".join([str(x) for x in list(chunks(df["id"].values, 5))[0]])

'730759428430008320,730759413938655232,730759400445644801,730759393965412352,730759392925261824'

In [63]:

result = twitter.lookup_status(id='730759400445644801,730759393965412352,730759392925261824,730759383253192705,730759377548918784')

for tw in result:
        result = db.tweets.insert_one({ "_id": str(tw["id"]), "tweet": tw} )

In [62]:
tw

{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu May 12 13:59:47 +0000 2016',
 'entities': {'hashtags': [{'indices': [42, 49], 'text': 'Brexit'},
   {'indices': [107, 118], 'text': 'StrongerIn'}],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 2365445227,
    'id_str': '2365445227',
    'indices': [3, 17],
    'name': 'European Mainstream',
    'screen_name': 'EU_Mainstream'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 730759377548918784,
 'id_str': '730759377548918784',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': True,
 'lang': 'en',
 'place': None,
 'quoted_status_id': 730733544989949953,
 'quoted_status_id_str': '730733544989949953',
 'retweet_count': 3,
 'retweeted': False,
 'retweeted_status': {'contributors': None,
  'coordinates': None,
  'created_at': 'Thu May 12 12:23:50 +0000 2016',
  'e

In [65]:
for filelist in list(chunks(onlyfiles, 100)):
    df_from_each_file = (pd.read_csv('Brexit_tweets_stance/' + f, sep='\t', header = None, names = ['id', 'sentiment']) for f in filelist)
    concatenated_df   = pd.concat(df_from_each_file, sort = False)
    
    for tweetchunk in list(chunks(concatenated_df["id"].values, 100)):
        query_string = ",".join([str(x) for x in tweetchunk])  

        try:
            result = twitter.lookup_status(id=query_string)
            for tw in result:
                result = db.tweets.insert_one({ "_id": str(tw["id"]), "tweet": tw} )

        except TwythonRateLimitError as e:
            # Wait if we hit the Rate limit
            wait_for_awhile()
        except:
            print(" FAILED: Unexpected error:", sys.exc_info()[0])
            pass

 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 'twython.exceptions.TwythonError'>
 FAILED: Unexpected error: <class 

In [66]:
logging.shutdown()
